In [1]:

source('../model_utils.R')
source('../snp_utils.R')

require('R.utils')
require('data.table')
require('tidyverse')
require('reshape2')
require('parallel')

Loading required package: MASS

Loading required package: Matrix

Loading required package: mnormt

Loading required package: qqman



For example usage please run: vignette('qqman')



Citation appreciated but not required:

Turner, S.D. qqman: an R package for visualizing GWAS results using Q-Q and manhattan plots. biorXiv DOI: 10.1101/005165 (2014).



Loading required package: rstan

Loading required package: StanHeaders

Loading required package: ggplot2

rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.7.1 (2016-02-15) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.22.0 (2018-04-21) successfully loaded. See ?R.oo for help.


Attac

In [32]:

list.tabs <- lapply(c(1,2,4,6,7), function(idx) 
    read.table(sprintf("../../../data/sig_snp_table_Testosterone_m2_%s.txt", idx), 
               header=TRUE, stringsAsFactors=FALSE))


In [35]:
# what are the counts for each
lapply(list.tabs, function(x) table(x$category))

[[1]]

  2   3   4 
161 651  18 

[[2]]

  2   3   4 
163 653  20 

[[3]]

  2   3   4 
149 539  20 

[[4]]

  2   3   4 
162 652  19 

[[5]]

  2   3   4 
163 652  20 


In [ ]:
# what is the overlap of each

In [44]:
f_vars <- lapply(list.tabs, function(x) (x %>% filter(category==2))$SNP)
m_vars <- lapply(list.tabs, function(x) (x %>% filter(category==3))$SNP)

In [42]:
length(intersect(f_vars[[1]], f_vars[[2]]))
length(intersect(f_vars[[1]], f_vars[[3]]))
length(intersect(f_vars[[1]], f_vars[[4]]))
length(intersect(f_vars[[1]], f_vars[[5]]))
length(intersect(f_vars[[2]], f_vars[[3]]))
length(intersect(f_vars[[3]], f_vars[[4]]))
length(intersect(f_vars[[4]], f_vars[[5]]))
length(intersect(f_vars[[3]], f_vars[[4]]))
length(intersect(f_vars[[3]], f_vars[[5]]))
length(intersect(f_vars[[4]], f_vars[[5]]))

[1] 161

[1] 148

[1] 161

[1] 161

[1] 148

[1] 148

[1] 162

[1] 148

[1] 148

[1] 162

In [45]:
length(intersect(m_vars[[1]], m_vars[[2]]))
length(intersect(m_vars[[1]], m_vars[[3]]))
length(intersect(m_vars[[1]], m_vars[[4]]))
length(intersect(m_vars[[1]], m_vars[[5]]))
length(intersect(m_vars[[2]], m_vars[[3]]))
length(intersect(m_vars[[3]], m_vars[[4]]))
length(intersect(m_vars[[4]], m_vars[[5]]))
length(intersect(m_vars[[3]], m_vars[[4]]))
length(intersect(m_vars[[3]], m_vars[[5]]))
length(intersect(m_vars[[4]], m_vars[[5]]))

[1] 649

[1] 537

[1] 650

[1] 649

[1] 536

[1] 537

[1] 651

[1] 537

[1] 536

[1] 651

In [2]:
load("../../../data/vary_priors_8/m2_fit_7.RData"); fit2 <- fit
load("../../../data/vary_priors_8/dat_Testosterone.RData")

In [4]:

    # fraction in non-null component
    p <- getPi(fit2)

    # sigmasq
    sigmasq <- getVars(fit2)
    Sigma <- getSigma(fit2)
 


In [15]:
sm_dat <- dat
sm_dat$dat$N <- 10000
sm_dat$chr <- sm_dat$chr[1:10000]
sm_dat$p <- sm_dat$p[1:10000]
sm_dat$snp <- sm_dat$snp[1:10000]

In [10]:
require('tictoc')

Loading required package: tictoc



In [19]:
    # assign each SNP to a category
tic() 
posterior.df <- posteriorSNPtable(dat, fit2)
toc()

173.976 sec elapsed


In [20]:
head(posterior.df)

,p1,p2,p3,p4,category,SNP
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,0.998922802192925,5.39882583914345e-05,0.00102311100854457,9.85401400616345e-08,1,rs6649864
2,0.998682704041611,5.6145962233044e-05,0.00126102270784195,1.2728831378525e-07,1,rs56328755
3,0.999241609447132,7.72288061218705e-05,0.000681068213538469,9.35332078674478e-08,1,rs28669107
4,0.99769551710652,0.000475727612874818,0.00182708396268701,1.67131791797568e-06,1,rs148672604
5,0.997785161315883,0.000171301159801894,0.00204286805940928,6.69464906608577e-07,1,rs146939120
6,0.999226625331786,0.000110355756249523,0.000662888469188568,1.30442775782964e-07,1,rs28510852


In [21]:
write.table(posterior.df, file=sprintf("%s/m2/snp_table_%s.txt", "../../../data/", "Testosterone_m2_7"), row.names=FALSE, quote=FALSE)


In [22]:

    cat.count <- table(posterior.df$category)
    print(cat.count)


     1      2      3      4 
335117    163    652     20 


In [24]:
p.df <- dat$p 
    se.df <- dat$dat$SE
    beta.df <- dat$dat$B
    snp.df <- data.frame(dat$snp, stringsAsFactors=FALSE)
    chr.df <- data.frame(dat$chr, stringsAsFactors=FALSE)
    full.df <- do.call(cbind, list(snp.df, chr.df, beta.df, se.df, p.df))
    colnames(full.df) <- c("SNP", "CHR", "B.f", "B.m", "SE.f", "SE.m", "P.f", "P.m")
    comb.df <- cbind(full.df, posterior.df %>% dplyr::select(p1, p2, p3, p4, category))
    non.null.snps <- comb.df %>% dplyr::filter(category %in% c(2,3,4))
write.table(non.null.snps, file=sprintf("%s/sig_snp_table_%s.txt", "../../../data/", "Testosterone_m2_7"), row.names=FALSE, quote=FALSE)


In [27]:
write.table(non.null.snps, file=sprintf("%s/sig_snp_table_%s.txt", "../../../data/", "Testosterone_m2_7"), row.names=FALSE, quote=FALSE)

In [26]:
    annot.snp <- annotateSNP(non.null.snps)

Warning message in file(file, "rt"):
“cannot open file '/scratch/PI/mrivas/users/erflynn/sex_div_gwas/data//snp_gene_table.txt': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
 print("Extracting")
    print(trait)


    # fraction in non-null component
    p <- getPi(fit2)

    # sigmasq
    sigmasq <- getVars(fit2)
    Sigma <- getSigma(fit2)
 
    # assign each SNP to a category
    posterior.df <- posteriorSNPtable(dat, fit2)
    write.table(posterior.df, file=sprintf("%s/m2/snp_table_%s.txt", out_dir, trait), row.names=FALSE, quote=FALSE)
    print("Posterior table generated")

    # remove large files from the workspace
    rm(fit2)

    cat.count <- table(posterior.df$category)
    print(cat.count)

    # create a data frame with the pvalues
    p.df <- dat$p 
    se.df <- dat$dat$SE
    beta.df <- dat$dat$B
    snp.df <- data.frame(dat$snp)
    chr.df <- data.frame(dat$chr)
    full.df <- do.call(cbind, list(snp.df, chr.df, beta.df, se.df, p.df))
    colnames(full.df) <- c("SNP", "CHR", "B.f", "B.m", "SE.f", "SE.m", "P.f", "P.m")
    comb.df <- cbind(full.df, posterior.df %>% dplyr::select(p1, p2, p3, p4, category))
    non.null.snps <- comb.df %>% dplyr::filter(category %in% c(2,3,4))

    annot.snp <- annotateSNP(non.null.snps)
     
    sapply(unique(annot.snp$category), function(category) {
        annot.snp.cat <- annot.snp %>% dplyr::filter(category==category)
        write.table(annot.snp.cat, file=sprintf("%s/m2/snps%s_%s.txt", out_dir, category, trait), row.names=FALSE)
         } )